# Анализ частотных существительных, прилагательных и глаголов

Считаем частотность всех указанных частей речи в каждом

Выводим топ 10 частотных слов на каждую часть речи

Анализируем общие слова из выведенных топ 10

In [85]:
import re
import pandas as pd
import numpy as np
from pymystem3 import Mystem
import string

import operator

import progressbar
import time

from collections import OrderedDict

In [91]:
def update_dict(word, pos_dict):
    """инкрементируем уже существующий элемент словаря или создаем несуществующий элемент со значением 1"""
    if(word not in pos_dict):
        pos_dict[word] = 1
    else:
        pos_dict[word] += 1
        
def verb_noun_adj_dict_creator(mystemmed_line, noun_dict, adj_dict, verb_dict):
    """ создаем словарь частотности глаголов, существительных и прилагательных """
    for mystemmed_word in mystemmed_line:
        keys = list(mystemmed_word.keys())
        values = list(mystemmed_word.values())
        word  = mystemmed_word['text']
        if ('analysis' not in keys):
            pass
        elif(mystemmed_word['analysis'] ==[]):
            pass
        else:
            grammar = mystemmed_word['analysis'][0]['gr']
            grammar_sep_by_comma = grammar.split(',')
            if (len(grammar_sep_by_comma) == 1):
                pass
            else:
                pos = re.match('[A-Z\s]+', grammar_sep_by_comma[0])[0]
                if(pos == "S"):                    
                    update_dict(values[-1], noun_dict)
                elif(pos == "A"):
                    update_dict(values[-1], adj_dict)
                elif(pos == "V"):
                    update_dict(values[-1], verb_dict)

def get_top_pos(ready_pos_dict,top_n):
    """сортируем словарь частотностей возвращаем top_n самых частотных слов"""
    top_pos_list = []
    sorted_dict_items = sorted(ready_pos_dict.items(), key=lambda kv: kv[1],reverse=True)
    top_words_count = min(len(ready_pos_dict), top_n)
    for word_index in range(top_words_count):
        top_pos_list.append(sorted_dict_items[word_index][0])
    return top_pos_list
                    
def verb_noun_adj_dict_creator_file(file):
    noun_dict = {}
    adj_dict ={}
    verb_dict = {}
    m = Mystem()
    line_number = 0
    with open (file, "r", encoding = "utf-8") as file:
        all_lines = file.readlines()
        bar = progressbar.ProgressBar(maxval=len(all_lines),
                                      widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
        bar.start()
        for line in all_lines:
            if(line != '\n' ):
                mystemmed_line = m.analyze(line)
                verb_noun_adj_dict_creator( mystemmed_line, noun_dict, adj_dict, verb_dict)
            line_number += 1
            bar.update(line_number)
            time.sleep(0.1)
    top_nouns = get_top_pos(noun_dict, 10)
    top_adj = get_top_pos(adj_dict, 10)
    top_verb = get_top_pos(verb_dict, 10)
    return top_nouns, top_adj, top_verb

#top_nouns_rzn, top_adj_rzn, top_verbs_rzn = verb_noun_adj_dict_creator_file('speech_rzn_processed.txt')

In [95]:
top_nouns_rzn,top_adj_rzn,top_verbs_rzn= verb_noun_adj_dict_creator_file('speech_rzn_processed.txt')

In [96]:
top_nouns_tmn, top_adj_tmn, top_verbs_tmn = verb_noun_adj_dict_creator_file('speech_tmn_processed.txt')

In [97]:
top_nouns_tula, top_adj_tula, top_verbs_tula = verb_noun_adj_dict_creator_file('speech_tula_processed.txt')

In [98]:
top_nouns_yml, top_adj_yml, top_verbs_yml = verb_noun_adj_dict_creator_file('speech_yml_processed.txt')

In [99]:
top_nouns_putin, top_adj_putin, top_verbs_putin = verb_noun_adj_dict_creator_file('speech_putin_processed.txt')

In [104]:
top_pos_words = {'ryazan':[top_nouns_rzn,top_adj_rzn,top_verbs_rzn],
             "tyumen":[top_nouns_tmn, top_adj_tmn, top_verbs_tmn ],
             "tula": [top_nouns_tula, top_adj_tula, top_verbs_tula], 
             "yamal": [top_nouns_yml, top_adj_yml, top_verbs_yml], 
             "putin":[top_nouns_putin, top_adj_putin, top_verbs_putin]}

putin_similarity_dict = {'ryazan':0,"tyumen":0,"tula": 0, "yamal": 0}
top_pos_words_ordered_dict = OrderedDict(top_pos_words)
keys = list(top_pos_words_ordered_dict.keys())

values = list(top_pos_words_ordered_dict.values())

In [105]:
def lists_intersection(lst1, lst2): 
    
    intersection = [value for value in lst1 if value in lst2] 
    return  intersection 

def verb_adj_nouns_intersection(first_top_w_list, second_top_w_list, putin):
    """сравниваем самые частотные части речи двух текстов, если сравниваем с речью Путина, 
    то возвращаем количесвто общих слов для далнейшего подсчет"""
    intersections = []
    common_words_count = 0
    for pos_top_words_index in range(3):
        pos_intersection = lists_intersection (first_top_w_list[pos_top_words_index], second_top_w_list[pos_top_words_index]) 
        intersections.append(pos_intersection)
    intersections_dict = {'nouns':intersections[0] ,'adjectives' : intersections[1], 'verbs' : intersections[2]}
    for pos, pos_intersection_list in intersections_dict.items():
        if(pos_intersection_list):
            common_words_count += len(pos_intersection_list)
            print("Common", pos, ":", pos_intersection_list)
        else:
            print("No common", pos)
    if (putin):
        return common_words_count

In [106]:
for dict_ind in range(len(keys)):
    current_top_w_list = values[dict_ind]
    for compare_ind in range(dict_ind + 1,len(keys)):
        print(keys[dict_ind], "vs", keys[compare_ind])
        compared_top_w_list = values[compare_ind]
        
        if(keys[compare_ind] != 'putin'):
            verb_adj_nouns_intersection(current_top_w_list, compared_top_w_list, putin = False)


        elif(keys[compare_ind] == 'putin'):
            common_words = verb_adj_nouns_intersection(current_top_w_list, compared_top_w_list, putin = True)
            putin_similarity_dict[keys[dict_ind]] = common_words

    print()
sorted_x = sorted(putin_similarity_dict.items(), key=operator.itemgetter(1))
print(sorted_x[-1][0],"is most similar to Putin speech")

ryazan vs tyumen
Common nouns : ['год', 'область', 'проект', 'человек', 'развитие']
Common adjectives : ['новый', 'должный', 'социальный', 'общественный', 'хороший']
Common verbs : ['сделать', 'становиться', 'обеспечивать', 'продолжать', 'создавать']
ryazan vs tula
Common nouns : ['год', 'регион', 'область', 'проект', 'развитие', 'работа']
Common adjectives : ['новый', 'должный', 'социальный', 'федеральный']
Common verbs : ['становиться', 'создавать']
ryazan vs yamal
Common nouns : ['год', 'регион', 'проект', 'развитие', 'работа']
Common adjectives : ['новый', 'социальный', 'федеральный']
Common verbs : ['становиться', 'продолжать']
ryazan vs putin
Common nouns : ['год', 'человек', 'развитие']
Common adjectives : ['новый', 'должный', 'современный']
Common verbs : ['позволять', 'сделать', 'становиться', 'обеспечивать', 'создавать']

tyumen vs tula
Common nouns : ['область', 'год', 'проект', 'развитие', 'гражданин']
Common adjectives : ['новый', 'должный', 'социальный', 'медицинский', 'р